<a class="anchor" id="python_data_model"></a>
## Python Data Model 
You might have wondered how the same built-in operator (`+, *`) or function (`len()`) shows different behavior for objects of different classes. This is called operator overriding or function overriding, respectively.

By default, most of the built-ins and operators will not work with objects of your classes. You must add the corresponding special methods in your class definition to make your object compatible with built-ins and operators. When you do this, the behavior of the function or operator associated with it changes according to that defined in the method.

This is exactly what the [Data Model (Section 3 of the Python documentation)](https://docs.python.org/3/reference/datamodel.html) helps you accomplish. It lists all the special methods available and provides you with the means of overriding built-in functions and operators so that you can use them on your own objects.

When you define these special methods in your own class, you override the behavior of the function or operator associated with them because, behind the scenes, Python is calling your method. 
In fact, when you obtain the list of attributes and methods of a str object using `dir()`, you’ll see these special methods in the list in addition to the usual methods available on `str` objects

1. **EXAMAPLE `__len__`**

```python

class Order:
    def __init__(self, cart, customer):
   #def __init__(self, *args, customer): (another way to implement)
        self.cart = list(cart)
        #self.items = args
        self.customer = customer
        
    def __len__(self):
        return len(self.cart)
    
order = Order(['banana', 'apple', 'mango'], 'Ali Hejazizo')
len(order)

----- ANS -----
3
```
As you can see, you can now use `len()` to directly obtain the length of the cart. Moreover, it makes more intuitive sense to say “length of order” rather than calling something like `order.get_cart_len()`. Your call is both Pythonic and more intuitive. When you don’t have the `__len__()` method defined but still call `len()` on your object, you get a `TypeError`.

But, when overriding `len()`, you should keep in mind that Python requires the function to return an integer. If your method were to return anything other than an integer, you would get a `TypeError`. This, most probably, is to keep it consistent with the fact that `len()` is generally used to obtain the length of a sequence, which can only be an integer.


2. **EXAMAPLE `__abs__`**

```python
class Vector:
    def __init__(self, x_comp, y_comp):
        self.x_comp = x_comp
        self.y_comp = y_comp

    def __abs__(self):
        return (self.x_comp ** 2 + self.y_comp ** 2) ** 0.5
    
    vector = Vector(3, 4)
    abs(vector)

----- ANS -----
5

```

3. **EXAMAPLE `__str__` and `__repr__`**
The `str()` built-in is used to cast an instance of a class to a str object, or more appropriately, to obtain a user-friendly string representation of the object which can be read by a normal user rather than the programmer. You can define the string format your object should be displayed in when passed to `str()` by defining the `__str__()` method in your class. Moreover, `__str__()` is the method that is used by Python when you call `print()` on your object.

```python
class Vector:
    def __init__(self, x_comp, y_comp):
        self.x_comp = x_comp
        self.y_comp = y_comp

    def __str__(self):
        # By default, sign of +ve number is not displayed
        # Using `+`, sign is always displayed
        return f'{self.x_comp}i + {self.y_comp}j'
    def __repr__(self):
        return (f'{self.__class__.__name__}({self.x_comp}, {self.y_comp})'
                
vector = Vector(3, 4)
print(vector)

----- ANS -----
3i+4j
```
Read [Python String Conversion 101: Why Every Class Needs a “repr”](https://dbader.org/blog/python-repr-vs-str) for a complete tutorial on `__repr__` and the difference between `__repr__` and `__str__`.
[“Difference between `__str__` and `__repr__` in Python”](https://stackoverflow.com/questions/1436703/what-is-the-difference-between-str-and-repr) is a Stack Overflow question with excellent contributions from Pythonistas Alex Martelli and Martijn Pieters.

>Rule of thumb: `__repr__` is for developers, `__str__` is for customers.


4. **EXAMAPLE `__bool__`**:

```python

class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __bool__(self):
        return len(self.cart) > 0
    
order1 = Order(['banana', 'apple', 'mango'], 'Ali Hejazizo')
order2 = Order([], 'Mohsen')

bool(order1), bool(order2)

for order in [order1, order2]:
    if order:
        print(f"{order.customer}'s order is processing...")
    else:
        print(f"Empty order for customer {order.customer}")

----- ANS -----
(True, False)

Ali Hejazizo's order is processing...
Empty order for customer Mohsen

```
> **Note:** When the `__bool__()` special method is not implemented in a class, the value returned by `__len__()` is used as the truth value, where a non-zero value indicates True and a zero value indicates `False`. In case both the methods are not implemented, all instances of the class are considered to be `True`.

5. **EXAMAPLE `__add__`/`+`**:

Changing the behavior of operators is just as simple as changing the behavior of functions. You define their corresponding special methods in your class, and the operators work according to the behavior defined in these methods.

These are different from the above special methods in the sense that they need to accept another argument in the definition other than `self`, generally referred to by the name `other`. Let’s look at a few examples.

```python
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __add__(self, other):
        new_cart = self.cart.copy()
        new_cart.append(other)
        return Order(new_cart, self.customer)
    
order = Order(["Apple"], "Ali")
(order + 'orange').cart  # New Order instance
order.cart  # Original instance unchanged
order = order + 'mango'  # Changing the original instance
order.cart

----- ANS -----
['Apple', 'orange']
['Apple']
['Apple', 'mango']
```
Similarly, you have the `__sub__()`, `__mul__()`, and other special methods which define the behavior of `-`, `*`, and so on. These methods should return a new instance of the class as well.

5. **EXAMAPLE `__getitem__`/Indexing/`[]`**:
The `[]` operator is called the indexing operator and is used in various contexts in Python such as getting the value at an index in sequences, getting the value associated with a key in dictionaries, or obtaining a part of a sequence through slicing. You can change its behavior using the `__getitem__()` special method.

```python
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer

    def __getitem__(self, key):
        return self.cart[key]
    
    def __setitem__(self, key, value):
        self.items[key] = value
    
order = Order(['banana', 'apple'], 'Ali Hejazizo')
order[-1]
order[:]

----- ANS -----
'apple'
['banana', 'apple']
```

**Note:** There is a similar `__setitem__()` special method that is used to define the behavior of `obj[x] = y`. This method takes two arguments in addition to self, generally called key and value, and can be used to change the value at key to value.

You’ll notice that above, the name of the argument to `__getitem__()` is not index but key. This is because the argument can be of mainly three forms:
- **an integer value**: in which case it is either an index or a dictionary key
- **a string value**: in which case it is a dictionary key
- **a slice object**: in which case it will slice the sequence used by the class. While there are other possibilities, these are the ones most commonly encountered.